# Chapter 9 thing I missed
* Explicitly, going over for the 200th time, the difference between constructors
* __copy constructor__ is used to initialize a _previously uninitialized_ object from some other object's data
* __assignment operator__ is used to replace the data of a _previously initalized_ object with some other object's data
* In c++11 you can explicitly default constructors with =default;
    * default constructor is copy-constructor

Related __rule of three__ "https://stackoverflow.com/questions/4172722/what-is-the-rule-of-three"
executive summary:
- "value semantics" == "objects are implicitly copied in various objects"
- "copying objects" == "values are assigned to an object that may or may not have had a state already(hence it picks diff handlers)
- the assignment b=a called "copy assignment operator" in this SE answer (is that right? isnt' is *just* the assignment operator?(it's called copy here because the values are copied, there is not a copy of the right-hand object being made))
    - this is bad because with pointers you get shallow copies(I think std containers do assignment copies fine)
Ok, so the rule of 3 at last:

```
    If you need to explicitly declare either the destructor, copy constructor or copy assignment operator yourself, you probably need to explicitly declare all three of them.

```

finally, these are also called _direct initialization_ and _copy initialization_
https://stackoverflow.com/questions/4293596/when-should-you-use-direct-initialization-and-when-copy-initialization
- Copy-initialization Type1 x = a;
- Direct-initialization Type1 x(a);
These are not equivalent, most notable in contexts where a conversion is required

## From section 9.5


In [1]:
struct wow{
    int a,b,c;
    int x;
    int *ayy;
}

In [2]:
#include <iostream>

In [3]:
wow b;  //checking default initialization
b.ayy = &b.x;
std::cout << b.x << "\n";
std::cout << b.b << "\n";
std::cout << b.ayy << "\n";
wow c(b);
std::cout << c.ayy << "\n";

wow d = b;
std::cout << d.ayy << "\n";

0
0
0x7f66450a1024
0x7f66450a1024
0x7f66450a1024


(std::basic_ostream<char, std::char_traits<char> > &) @0x7f66403a3f40


In [4]:
int &t = b.x;
std::cout << t << "\n";
std::cout << &t << "\n";
std::cout << &b.x << "\n";

0
0x7f66450a1024
0x7f66450a1024


(std::basic_ostream<char, std::char_traits<char> > &) @0x7f66403a3f40


# Chapter 10 thing I probably didn't miss but am writing down for posterity

* in a declaration like `int *p`, the `*p` part is called the "declarator", i.e. `*p` is a *declarator*
* it's most often said in the above statement, that `p` has type `int*`, but it would be more correct to say that `*p` has type int
* function ptr syntax review : `int (*myfp)(int)` creates myfp which holds a function that returns an int and accepts an int
    * `fp = next`, `fp = &next` are equivalent, and `i = (*fp)(i)`, `i = fp(i)` are equivalent

* stop forgetting, new T[20]; needs to be freed with delete[] T;

In [5]:
// This function is completely legitimate. The author refers to this as "static memory management"
int* pointer_to_static()
{
    static int x;  //the keyword
    return &x;
}
std::cout << pointer_to_static() << "\n";
std::cout << pointer_to_static() << "\n";

*pointer_to_static() = 23;
std::cout << *pointer_to_static();


0x7f66450a1070
0x7f66450a1070
23

(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f66403a3f40


In [6]:
int* p = new int(42);
std::cout << p << "\n";
std::cout << *p << "\n";
std::cout << ++p << "\n";
std::cout << *p << "\n";

0x7f6610a75750
42
0x7f6610a75754
32614


(std::basic_ostream<char, std::char_traits<char> > &) @0x7f66403a3f40


In [7]:
int* o = new int[42];

In [8]:
std::cout << *o << "\n";
std::cout << *(o+1) << "\n";
std::cout << *(o+2) << "\n";
std::cout << *(o+3) << "\n";
std::cout << *(o+4) << "\n";
std::cout << *(o+5) << "\n";
std::cout << *(o+6) << "\n";
std::cout << *(o+7) << "\n";

268435736
32614
268435736
32614
670997700
32614
5
32614


(std::basic_ostream<char, std::char_traits<char> > &) @0x7f66403a3f40


# Chapter 11 things I am altruisticly recording for future generations

oh man I totally got sidetracked in the other window and now the closing music is on r i p

overall it was fun, but I didn't focus to the level that I think really effective people do, but focusing too much also has downsides(i.e. getting fixated on the wrong things), which can be curtailed in their own way. Anyway, I'm going to close the stream here, and continue this tomorrow hopefully with some more real examples.

* explicit keyword for constructors : says the compiler will use the constructor only when the user expressly invokes the constructor (i.e. the implicit = thing won't use that constructor(only makes sense in constructor with single arg))

```
//where constructor is set as explicit
Vec<int> vi(100);  // ok, explicitly construct the Vec from an int
Vec<int> vi = 100;  // error: implicitly construct the Vec (§11.3.3/199) and copy
```

## 11.3 copy control (i.e. the C++ gestapo)

```
C++ is the only language in widespread use that gives the programmer this level of control
over an object's behavior. Not surprisingly, getting these operations correct is essential in
building useful data types.
```
^this quote makes me feel slight power


### Copy constructor
Passing an object by value to a function, or returning an object *by value* from a function implicitly copies the object, hence calling the copy constructor to make the new implicit object.

explicit version:
```
vector<Student_info> vs;
vector<Student_info> v2 = vs;  // copy vs into v2
```
\-> copy constructor should be a const reference and reference type, but since we're defining what it means to make a copy, we can just screw it up

Is self-assignment really such a big deal to consider?
just remember, initialization != assignment


## 11.5 flexible memory management
* We don't want to use new and delete because it would make our type restrictive
  * `new` both allocates *and* initializes memory>. When constructing a vector of type T, it needs the default constructor for T. Initializing every element can also be expensive and we may in-fact want to defer doing so in many cases!
* Instead of using new and delete, we can use &lt;memory&gt;
  * provides class called `allocator<T>`, that allocates a block of uninitialized memory, this way we can choose what constructor we want (how would this work exactly?)
  
4 relevant methods

```
T* allocate(size_t);
void deallocate(T*, size_t);
void construct(T*, const T&) ;
void destroy(T*);
```

there is also `uninitialized_copy` and `uninitialized_fill`
* uninitialized_fill, copies third argument into seq of uninitialized elements
* uninitialized_copy, copies elements from source sequence to a target sequence of uninitialized elements

`ptrdiff_t` is a thing [reminder]

In [9]:
struct copy_me{  //pls i need it
  public:
    copy_me() = default;   //ignoring the rule of 3 here!!
    copy_me(copy_me &other){  //healthy side-effects
      x = other.x;
      y = other.y;
      ++other.x;
      ++other.y;
    }
    int x, y;
};

copy_me r;
r.x = 23; r.y = 2;
std::cout << "Before\n";
std::cout << r.x << "  " << r.y << "\n";
copy_me r_o = r;
std::cout << "After\n";
std::cout << r.x << "  " << r.y << "\n";
std::cout << r_o.x << "  " << r_o.y << "\n";

Before
23  2
After
24  3
23  2


(std::basic_ostream<char, std::char_traits<char> > &) @0x7f66403a3f40


In [10]:
struct copy_me_2{  //pls i need it
  public:
    copy_me_2() = default;   //ignoring the rule of 3 here!!
    copy_me_2(copy_me_2 &other){  //healthy side-effects
      x = other.x;
      y = other.y;
      ++other.x;
      ++other.y;
    }
    copy_me_2& operator=(copy_me_2& other){  //not sure what this return type means, clarify w/ chris
      //check for self-assignment
      if (&other != this){
        x = other.x;
        y = other.y;
        --other.x;
        --other.y;
      }
      return (*this);  //we have to return an object here
    }
    int x, y;
};

copy_me_2 e, e_l;
e.x = 4; e.y = 14;
std::cout << "Before\n";
std::cout << e.x << "  " << e.y << "\n";
copy_me_2 e_o = e;

std::cout << "After\n";
std::cout << e.x << "  " << e.y << "\n";
std::cout << e_o.x << "  " << e_o.y << "\n";

e_l = e;

std::cout << "More After\n";
std::cout << e.x << "  " << e.y << "\n";
std::cout << e_l.x << "  " << e_l.y << "\n";

Before
4  14
After
5  15
4  14
More After
4  14
5  15


(std::basic_ostream<char, std::char_traits<char> > &) @0x7f66403a3f40


In [12]:
#include <memory>

In [14]:
template <class T> class Vec {
public:
  typedef T* iterator;
  typedef const T* const_iterator;
  typedef size_t size_type;
  typedef T value_type;
  Vec() { create(); }
  explicit Vec(size_type n, const T& t = T()) { create(n, t); }
  Vec(const Vec& v) { create(v.begin(), v.end()); }
  Vec& operator=(const Vec&);
  // as defined in §11.3.2/196
  ~Vec() { uncreate(); }
  T& operator[](size_type i) { return data[i]; }
  const T& operator[](size_type i) const { return data[i]; }
  void push_back(const T& t) {
    if (avail == limit)
    grow();
    unchecked_append(t);
  }
  size_type size() const { return avail - data; }
  iterator begin() { return data; }
  const_iterator begin() const { return data; }
  iterator end() { return avail; }
  const_iterator end() const { return avail; }
private:
  iterator data; // first element in the Vec
  iterator avail; // (one past) the last element in the Vec
  iterator limit; // (one past) the allocated memory
  // facilities for memory allocation
  std::allocator<T> alloc;
  // object to handle memory allocation
  // allocate and initialize the underlying array  
  void create() ;
  void create(size_type, const T&);
  void create(const_iterator, const_iterator);
  // destroy the elements in the array and free the memory
  void uncreate();
  // support functions for push_back
  void grow();
  void unchecked_append(const T&);
};